In [4]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import chain, combinations
from scipy.linalg import det

In [1]:
def bernoulli_trial(p):
    return np.random.rand() < p

def poulson_dpp_sampling(K):
    n = K.shape[0]
    sample = []
    A = np.array(K, dtype=float)

    for j in range(n):
        if bernoulli_trial(A[j, j]):
            sample.append(j+1)
        else:
            A[j, j] -= 1 
        A[j+1:n-1, j] /= A[j, j]  
        A[j+1:n-1, j+1:(n-1)] -= np.outer(A[j+1:(n-1), j], A[j, j+1:(n-1)])
    return sample, A


In [2]:
sample, _ = poulson_dpp_sampling(K)
sample

NameError: name 'K' is not defined

In [5]:
# Generate a random 3x3 matrix A
A = np.random.randn(3, 3)
K = A @ A.T  
eigenvalues, _ = np.linalg.eig(K)
largest_eigenvalue = max(eigenvalues)
K = K/ (2 * largest_eigenvalue)
I = np.eye(3)
L = K @ np.linalg.inv(I - K)
print("---K Matrix---")
print(K)
print("---L Matrix---")
print(L)


---K Matrix---
[[0.45231249 0.12403444 0.01956838]
 [0.12403444 0.17136054 0.00206365]
 [0.01956838 0.00206365 0.02666511]]
---L Matrix---
[[0.89137616 0.28320578 0.03862557]
 [0.28320578 0.24920974 0.00834226]
 [0.03862557 0.00834226 0.02818985]]


In [6]:
def powerset(iterable):
    "powerset([1,2,3]) --> [], [1], [2], [3], [1,2], [1,3], [2,3], [1,2,3]"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [7]:
# Calculate the powerset of indices, including the empty set
indices = [1, 2, 3]
ps = list(powerset(indices))
ps

[(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

In [10]:
det_submatrices = {p: (det(K[np.ix_([i-1 for i in p], [j-1 for j in p])]) if p else 1.0) for p in ps}
print(det_submatrices.keys())
print("---Det of K---")
for subset, determinant in det_submatrices.items():
    subset_str = ", ".join(map(str, subset))  
    print(f"Subset ({subset_str}): {determinant}")
print("----Probabilities using L Matrix---")
probs = {p: (det(L[np.ix_([i-1 for i in p], [j-1 for j in p])]) if p else 1.0)/det(I+L) for p in ps}

sorted_probabilities = sorted(probs.items(), key=lambda x: (len(x[0]), x[0]))
print("Total Probabilities: " + str(sum(probs.values())))
for subset, probability in sorted_probabilities:
    subset_formatted = f"({', '.join(map(str, subset))})" if subset else "()"
    print(f"Subset {subset_formatted}: {probability:.6f}")


dict_keys([(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)])
---Det of K---
Subset (): 1.0
Subset (1): 0.45231248544535557
Subset (2): 0.17136053997179693
Subset (3): 0.026665112764954927
Subset (1, 2): 0.06212397061222356
Subset (1, 3): 0.01167804175632243
Subset (2, 3): 0.0045650894645250135
Subset (1, 2, 3): 0.001599016371008714
----Probabilities using L Matrix---
Total Probabilities: 1.0
Subset (): 0.426430
Subset (1): 0.380109
Subset (2): 0.106270
Subset (3): 0.012021
Subset (1, 2): 0.060525
Subset (1, 3): 0.010079
Subset (2, 3): 0.002966
Subset (1, 2, 3): 0.001599


In [9]:
def DPP_sampling(iterations):
    final_results = defaultdict(int)

    for _ in range(iterations):
        sample, _ = poulson_dpp_sampling(K) 
        final_results[tuple(sample)] += 1

    probabilities = {subset: count / iterations for subset, count in final_results.items()}
    sorted_probabilities = dict(sorted(probabilities.items(), key=lambda item: (len(item[0]), item)))
    print("Total Probabilities: " + str(sum(probabilities.values())))
    for subset, probability in sorted_probabilities.items():
        subset_formatted = f"({', '.join(map(str, subset))})" if subset else "()"
        print(f"Subset {subset_formatted}: {probability:.6f}")

    return [probability for subset, probability in sorted_probabilities.items()]

iterations = 10000000
probabilities = DPP_sampling(iterations)

Total Probabilities: 1.0
Subset (): 0.426760
Subset (1): 0.379822
Subset (2): 0.106348
Subset (3): 0.011677
Subset (1, 2): 0.060454
Subset (1, 3): 0.010388
Subset (2, 3): 0.002912
Subset (1, 2, 3): 0.001638
